In [3]:
# libraries importation

import pandas as pd

In [6]:
news = pd.read_csv('./data/fisrt_news.csv', names = ['title', 'text', 'author', 'url'])

In [7]:
news.head()

,title,text,author,url
0,La caprichosa y millonaria primera decisión de...,"Begoña Gómez quiere cambios más ""contundentes""...",SALOMON LUSH,https://www.periodistadigital.com/periodismo/t...
1,Santiago Abascal dice que perdona a las manife...,Santiago Abascal se ha mostrado generoso al op...,Ayudante de Becario,https://lavozdelbecario.es/santiago-abascal-di...
2,El aborto será legal durante todo el embarazo ...,La norma avala el aborto tardío en cualquier m...,Actuall / Aci Prensa,https://www.actuall.com/vida/el-aborto-sera-le...
3,El Gobierno cambiará los cementerios españoles...,La naturaleza y la composición de los ritos fu...,Mediterráneo Digital,https://www.mediterraneodigital.com/espana/and...
4,"Dos perros ""contraen matrimonio""","La concejal de Seguridad de Lorca, Murcia (Esp...",Redacción ACI Prensa,https://www.aciprensa.com/noticias/casan-a-dos...


In [24]:
news['label'] = True 


In [25]:
news.loc[:59,'label'] = False

True     60
False    60
Name: label, dtype: int64